# Aprendizaje automático relacional

#### Fernando Jesús Fernández Gallardo
#### Carmen Galván López

# Librerías externas y variables globales

In [ ]:
import networkx as nx
from pandas import read_csv
from numpy import mean
from sklearn import preprocessing, model_selection, naive_bayes, neighbors
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from scipy import stats
from copy import copy
# Suprimir los warnings relacionados con la siguiente versión de SciPy
from warnings import filterwarnings

semilla = 86
test_size= .33
filterwarnings("ignore", category=FutureWarning)
filterwarnings("ignore", category=UserWarning)
cross_val = None
codificador_objetivo = None

# Definición de los modelos
### Naive Bayes

In [ ]:
"""
Ejecuta el modelo Naive Bayes sobre el dataset, tanto para datos no relacionales como
relacionales

@param tuple(atributos_entrenamiento, atributos_prueba, atributos_codificados) - Variables de los atributos a utilizar
@param tuple(objetivo_entrenamiento, objetivo_prueba, objetivo_codificado) - Variables del objetivo a utilizar
@param suavizado = 1 - Suavizado de Laplace a aplicar
"""
def nb(atrs, obj, suavizado = 1):
    global codificador_objetivo, cv
    (at_ent, at_pr, at_cod) = atrs
    (ob_ent, ob_pr, ob_cod) = obj
    clasif_NB = naive_bayes.MultinomialNB(alpha=suavizado)
    clasif_NB.fit(at_ent, ob_ent)

    #Calculamos la cantidad de ejemplos para cada clase y los logaritmos
    for clase, cantidad_ejemplos_clase, log_probabilidad_clase in zip(
            clasif_NB.classes_, clasif_NB.class_count_, clasif_NB.class_log_prior_):
        print(f"Cantidad de ejemplos para la clase {clase}: {cantidad_ejemplos_clase}")
        print(f"Logaritmo de la probabilidad aprendida para la clase {clase}: {log_probabilidad_clase}")
    
    print("\nRESULTADOS DEL ENTRENAMIENTO")
    #Probamos la predicción con los atributos de prueba
    print(f'Predicción con Naive Bayes: {codificador_objetivo.inverse_transform(clasif_NB.predict(at_pr))}')
    #Hacemos el score con naive bayes
    print(f'Precisión con Naive Bayes: {clasif_NB.score(at_pr, ob_pr)}')
    #Hacemos el score con cross validation
    print(f'Precisión con cross validation: {cross_val_score(clasif_NB, at_cod, ob_cod, cv=cv)}')
    #Hacemos la media de score de cross validation, ya que al final es lo que nos interesa
    print(f'Media de precisión: {mean(cross_val_score(clasif_NB, at_cod, ob_cod, cv=cv))}')

### kNN

In [ ]:
"""
@param tuple(atributos_entrenamiento, atributos_prueba, atributos_codificados) - Variables de los atributos a utilizar
@param tuple(objetivo_entrenamiento, objetivo_prueba, objetivo_codificado) - Variables del objetivo a utilizar
"""
def knn(atrs, obj, ejemplos):
    global codificador_objetivo, cv
    (at_ent, at_pr, at_cod) = atrs
    (ob_ent, ob_pr, ob_cod) = obj

    def encontrar_mejor_k(at_cod, ob_cod, k_range):
        puntajes_por_k = []

        for k in k_range:
            # Crear clasificador KNN con el valor actual de k
            knn = KNeighborsClassifier(n_neighbors=k)

            # Realizar validación cruzada y obtener los puntajes
            puntajes = cross_val_score(knn, at_cod, ob_cod, scoring='accuracy', cv=cv)

            # Calcular el puntaje medio de validación cruzada
            puntaje_medio = puntajes.mean()

            # Almacenar el puntaje correspondiente al valor de k
            puntajes_por_k.append((k, puntaje_medio))

        return puntajes_por_k

    encontrar_mejor_k(at_cod, ob_cod, k_range=list(range(1, 20)))
    
    #Según el k_scores, podemos ver que a partir de 10 vecinos es totalmente irrelevante cuantos pongamos, 
    #así que utilizaremos n_neighbors=10,
    #Definimos y entrenamos kNN

    clasif_kNN = neighbors.KNeighborsClassifier(n_neighbors=10, metric='hamming')

    clasif_kNN.fit(at_ent, ob_ent)

    # Probamos la predicción con los atributos de prueba
    print('Predicción kNN:', codificador_objetivo.inverse_transform(clasif_kNN.predict(at_pr)))

    # Hacemos el score con kNN
    print('Precisión kNN:', clasif_kNN.score(at_pr, ob_pr))

    # Hacemos el score con cross validation utilizando los datos de entrenamiento
    cv_scores = cross_val_score(clasif_kNN, at_ent, ob_ent, cv=cv)
    print('Precisión cross validation:', mean(cv_scores))
    
    """
    Vamos a dar algunos datos sobre knn como pueden ser la distancia, los vecinos más cercanos a un dato,
    y las clases de esos vecinos
    """
    distancias, vecinos = clasif_kNN.kneighbors(ejemplos)
    print("Primer ejemplo nuevo:", ejemplos[0])
    print("10 vecinos más cercanos:")
    print([vecinos[0]])
    print("Distancias a esos vecinos (cantidad de atributos con valores distintos / cantidad total de atributos):")
    print(distancias[0])

    print("Clases a las que pertenecen esos vecinos: ", [objetivo[index] for index in vecinos[0]])

### SVC

In [ ]:
"""
@param tuple(atributos_entrenamiento, atributos_prueba, atributos_codificados) - Variables de los atributos a utilizar
@param tuple(objetivo_entrenamiento, objetivo_prueba, objetivo_codificado) - Variables del objetivo a utilizar
"""
def svc(atrs, obj):
    global cv
    (at_ent, at_pr, at_cod) = atrs
    (ob_ent, ob_pr, ob_cod) = obj
    classif_SVC = SVC().fit(at_ent, ob_ent)

    #Probamos la predicción con los atributos de prueba
    print(f'Predicción SVC: {codificador_objetivo.inverse_transform(classif_SVC.predict(at_pr))}')
    #Hacemos el score con kNN
    print(f'Precisión SVC: {classif_SVC.score(at_pr, ob_pr)}')
    #Hacemos el score con cross validation
    print(f'Precisión cross validation: {mean(cross_val_score(classif_SVC, at_cod, ob_cod, cv=cv))}')

# Procesamiento inicial de los datos en bruto
#### Lectura y eliminación del identificador

In [ ]:
#Leemos los archivos
vertices = read_csv('data/political-books-nodes.csv')
aristas = read_csv('data/political-books-edges.csv')

del(vertices['Id'])

#Mostramos las primeras 35 filas
vertices.head(35)

#### Validación y selección de los datos brutos

In [ ]:
"""
Comprobamos que el dataset es válido verificando que no existen duplicados
"""
if len(vertices) != len(set(vertices['Label'])):
    raise ValueError("El dataset no es válido ya que contiene duplicados")
"""
La mejor forma de identificar cada uno de los elementos que forma parte
del conjunto de entrenamiento es el nombre del propio libro (que en el dataset
se llama 'Label') en vez del ID o cualquier otro tipo de indentificador más
complejo. De esta forma, también es más fácil identificar elementos duplicados
(si los hubiera)
"""
atributos = vertices['Label']
"""
Nuestro objetivo es predecir la ideología política del autor basándonos en
sus obras, por lo que el objetivo que perseguimos en nuestro modelo
es el de la ideología política
"""
objetivo = vertices['political_ideology']

#### Codificación de los datos e inicialización de la CrossValidation

In [ ]:
"""
Para poder trabajar con los datos que tenemos, necesitamos convertirlos en un formato que sklearn pueda "entender".
Debemos de hacer que nuestros datos "planos" sean para sklearn objetos "comparables", dependiendo del tipo de
ordenación que nosotros veamos más apropiada para el método en cuestión
(de una manera similar hacemos en Java cuando implementamos la interfaz 'Comparable' y el método compareTo)

El codificador adecuado para la variable objetivo es LabelEncoder, que trabaja
con una lista o array unidimensional de sus valores y admite cadenas

"""
# Codificadores
codificador_atributos = preprocessing.LabelEncoder()
codificador_objetivo = preprocessing.LabelEncoder()
# Datos codificados
atributos_codificados = codificador_atributos.fit_transform(atributos)
objetivo_codificado = codificador_objetivo.fit_transform(objetivo)

"""
ShuffleSplit es necesario para la CrossValidation
"""
cv = ShuffleSplit(n_splits=10, test_size=test_size, random_state=semilla)

# Métricas no relacionales
#### División en conjunto de entrenamiento y conjunto de prueba

Partimos el atributo y el objetivo en dos conjuntos: entrenamiento y prueba.<br />
El de entrenamiento lo utilizaremos para la ejecución de los algoritmos, mientras que el de prueba sirve para evaluar su rendimiento

In [ ]:
(atributos_entrenamiento,
 atributos_prueba,
 objetivo_entrenamiento,
 objetivo_prueba) = model_selection.train_test_split(
        atributos_codificados,
        objetivo_codificado,
        # Valor de la semilla aleatoria para que el muestreo sea reproducible a pesar de ser aleatorio
        random_state=semilla,
        test_size=test_size,
        stratify=objetivo_codificado
)

El método reshape solo cambia la forma del array, pero no su contenido.
Los clasificadores de sklearn espera un array 2D porque puede manejar
múltiples características por muestra, pero en nuestro caso solo tenemos una característica por muestra,
que es el nombre del libro.

In [ ]:
atr_cod_reshaped = atributos_codificados.reshape(-1, 1)
atr_pr_reshaped = atributos_prueba.reshape(-1, 1)
atr_ent_reshaped = atributos_entrenamiento.reshape(-1, 1)

Ejecutamos los modelos:

In [ ]:
print("======= NAIVE BAYES (no relacional) =======\n")
nb((atr_ent_reshaped, atr_pr_reshaped, atr_cod_reshaped),
           (objetivo_entrenamiento, objetivo_prueba, objetivo_codificado))

print("\n\n======= kNN (no relacional) =======\n")
# Otorgamos a kNN nuevos ejemplos arbitrarios sobre los que aplicar el algoritmo entrenado para ver su clasificación
ejemplos_knn = [[1.], [20.], [2.]]
knn((atr_ent_reshaped, atr_pr_reshaped, atr_cod_reshaped),
           (objetivo_entrenamiento, objetivo_prueba, objetivo_codificado), ejemplos_knn)

print("\n\n======= SVC (no relacional) =======\n")
svc((atr_ent_reshaped, atr_pr_reshaped, atr_cod_reshaped),
           (objetivo_entrenamiento, objetivo_prueba, objetivo_codificado))

# Métricas relacionales
Definimos las funciones que vamos a utilizar para obtener las diferentes métricas relacionales

In [ ]:
"""
Métrica relacionada con la centralidad de un grafo. Devuelve el número de conexiones (enlaces) que tiene ese nodo.
Los nodos con mayor grado se consideran más centrales en términos de conectividad.
"""
def getDegreeCentrality(grafo):
    centrality = nx.degree_centrality(grafo)
    return list(centrality.values())
"""
Métrica relacionada con la centralidad. Si un nodo tiene un mayor grado,
significa que está conectado a un mayor número de otros nodos en el grafo.
"""
def getHighClosenessCentrality(grafo):
    high_closeness = nx.closeness_centrality(grafo, u=None, distance=None, wf_improved=True)
    return list(high_closeness.values())
"""
Métrica relacionada con la centralidad.
Un nodo con un alto valor de "high betweenness centrality" actúa como un puente o un punto de conexión
crucial entre diferentes partes del grafo.
"""
def getHighBetweennessCentrality(grafo):
    betweenness = nx.betweenness_centrality(grafo, normalized=True, endpoints=False)
    return list(betweenness.values())
"""
Es una medida que cuantifica qué tan conectados están los vecinos
de un nodo en comparación con todas las posibles conexiones entre ellos.
"""
def getCoeficienteClustering(grafo):
    # Obtener el coeficiente de clustering para cada vértice
    coeficientes = nx.clustering(grafo)
    return list(coeficientes.values())
"""
EXTRA: Modularidad. Esta métrica no nos es útil para ninguno de los algoritmos de ML que vamos
a utilizar, pero lo definimos para tener aún más información sobre el dataset que estamos estudiando.

Métrica relacionada con la detección de comunidades. Si un nodo tiene un mayor grado,
significa que está conectado a un mayor número de otros nodos en el grafo.
"""
def getModularidad(grafo):
    particion = nx.community.greedy_modularity_communities(grafo)
    modularidad = nx.community.modularity(grafo, particion)
    return modularidad

### Cálculo de las métricas relacionales
Generamos el grafo y añadimos las métricas al conjunto de vértices:

In [ ]:
grafo = nx.from_pandas_edgelist(aristas, 'Source', 'Target')
vertices_r = copy(vertices)

vertices_r = vertices_r.assign(Degree_Centrality = getDegreeCentrality(grafo))
vertices_r = vertices_r.assign(High_closeness_centrality = getHighClosenessCentrality(grafo))
vertices_r = vertices_r.assign(High_betweenness_centrality = getHighBetweennessCentrality(grafo))
vertices_r = vertices_r.assign(Clustering = getCoeficienteClustering(grafo))

print(f"La modularidad del grafo es: {getModularidad(grafo)}")

G = nx.read_graphml('data/political-books-network.graphml')
nx.draw_random(G, with_labels=True)
vertices_r.head(25)

## Selección de datos relacional

In [ ]:
#División de atributos y target
atributos_r = vertices_r[['Label', 'Degree_Centrality', 'High_closeness_centrality',
                     'High_betweenness_centrality','Clustering']]

atributos_r.loc[:, 'Label'] = atributos_codificados

#División de los datos en entreamiento y objetivo
(atributos_entrenamiento_r,
 atributos_prueba_r,
 objetivo_entrenamiento_r,
 objetivo_prueba_r) = model_selection.train_test_split(
        atributos_r,
        objetivo_codificado,
        # Valor de la semilla aleatoria para que el muestreo sea reproducible a pesar de ser aleatorio
        random_state=semilla,
        test_size=test_size,
        stratify=objetivo_codificado
)

Ejecutamos los modelos nuevamente, esta vez con los datos relaciones:

In [ ]:
print("======= NAIVE BAYES (relacional) =======\n")
nb((atributos_entrenamiento_r, atributos_prueba_r, atributos_r),
           (objetivo_entrenamiento_r, objetivo_prueba_r, objetivo_codificado))

print("\n\n======= kNN (relacional) =======\n")
ejemplos_knn_r = [[1., 0.23, 0.4321, 0.001, 0.333333], [20., 0.115, 0.3095, 0.00521, 0.5], [2., 0.0579, 0.3151, 0.0099, 0.6]]
knn((atributos_entrenamiento_r, atributos_prueba_r, atributos_r),
           (objetivo_entrenamiento_r, objetivo_prueba_r, objetivo_codificado), ejemplos_knn_r)

print("\n\n======= SVC (relacional) =======\n")
svc((atributos_entrenamiento_r, atributos_prueba_r, atributos_r),
           (objetivo_entrenamiento_r, objetivo_prueba_r, objetivo_codificado))